In [ ]:
'''
https://stackoverflow.com/questions/64272731/open-cv-shows-green-screen-on-jetson-nano
https://forums.developer.nvidia.com/t/green-screen-when-using-raspberry-pi-camera-v2-attached-to-jetson-nano-and-cv2-opencv/173596/9
https://qengineering.eu/install-opencv-4.5-on-jetson-nano.html
https://gist.github.com/pohmelie/cf4eda5df24303325b16
https://blog.csdn.net/kingroc/article/details/93893828

完整教學看這兩個
https://qengineering.eu/install-opencv-4.5-on-jetson-nano.html
https://www.sproutworkshop.com/2021/04/how-to-compile-opencv-4-5-2-with-cuda-11-2-and-cudnn-8-1-on-ubuntu-20-04/
'''


'''
# 注意，OPENCV_EXTRA_MODULES_PATH 路徑要給對！
# 跑出來的結果要檢查 python3 的部分路徑有沒有抓到

cmake -D CMAKE_BUILD_TYPE=RELEASE \
-D CMAKE_INSTALL_PREFIX=/usr \
-D OPENCV_EXTRA_MODULES_PATH=../../opencv_contrib/modules \
-D EIGEN_INCLUDE_PATH=/usr/include/eigen3 \
-D WITH_OPENCL=OFF \
-D WITH_CUDA=ON \
-D CUDA_ARCH_BIN=5.3 \
-D CUDA_ARCH_PTX="" \
-D WITH_CUDNN=ON \
-D WITH_CUBLAS=ON \
-D ENABLE_FAST_MATH=ON \
-D CUDA_FAST_MATH=ON \
-D OPENCV_DNN_CUDA=ON \
-D ENABLE_NEON=ON \
-D WITH_QT=OFF \
-D WITH_OPENMP=ON \
-D WITH_OPENGL=ON \
-D BUILD_TIFF=ON \
-D WITH_FFMPEG=ON \
-D WITH_GSTREAMER=ON \
-D WITH_TBB=ON \
-D BUILD_TBB=ON \
-D BUILD_TESTS=OFF \
-D WITH_EIGEN=ON \
-D WITH_V4L=ON \
-D WITH_LIBV4L=ON \
-D OPENCV_ENABLE_NONFREE=ON \
-D INSTALL_C_EXAMPLES=OFF \
-D INSTALL_PYTHON_EXAMPLES=OFF \
-D BUILD_opencv_python3=TRUE \
-D OPENCV_GENERATE_PKGCONFIG=ON \
-D BUILD_EXAMPLES=OFF ..
'''

'''
# 重點！！！！
make install 前先把預設的 python ln 到 python3.6 (system 的)
make install 後，自行把 cv2.so link 到 virtualenv
'''

import cv2
# from  matplotlib import pyplot as plt
import time
%matplotlib inline

In [ ]:
def gstreamer_pipeline(
    capture_width=1280,
    capture_height=720,
    display_width=1280,
    display_height=720,
    framerate=60,
    flip_method=0,
):
    return (
        "nvarguscamerasrc ! "
        "video/x-raw(memory:NVMM), "
        "width=(int)%d, height=(int)%d, "
        "format=(string)NV12, framerate=(fraction)%d/1 ! "
        "nvvidconv flip-method=%d ! "
        "video/x-raw, width=(int)%d, height=(int)%d, format=(string)BGRx ! "
        "videoconvert ! "
        "video/x-raw, format=(string)BGR ! appsink"
        % (
            capture_width,
            capture_height,
            framerate,
            flip_method,
            display_width,
            display_height,
        )
    )

In [ ]:
# from jetbot source code
def _gst_str(width=224, height=224, fps=30, capture_width=816, capture_height=616):
        return 'nvarguscamerasrc sensor-mode=3 ! video/x-raw(memory:NVMM), width=%d, height=%d, format=(string)NV12, framerate=(fraction)%d/1 ! nvvidconv ! video/x-raw, width=(int)%d, height=(int)%d, format=(string)BGRx ! videoconvert ! appsink' % (
                capture_width, capture_height, fps, width, height)

In [ ]:
print(gstreamer_pipeline())
print(_gst_str())

In [ ]:
gst = _gst_str()
# gst = gstreamer_pipeline()
cap = cv2.VideoCapture(gstreamer_pipeline(), cv2.CAP_GSTREAMER)

In [ ]:
cap.isOpened()

In [ ]:
print('test speed')
t = time.time()
success, img = cap.read()
print(time.time() -t)

In [9]:
from IPython.display import clear_output, display
from PIL import Image
def render(img_arr, fmt='jpeg'):
    img = Image.fromarray(img_arr)
    display(img)
    clear_output(wait=True)

In [10]:
while True:
    t = time.time()
    success, img = cap.read()
    if success:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        render(img)

    else:
        print('camera error')

KeyboardInterrupt: 

In [ ]:
cap.release()

In [6]:
print(cv2.getBuildInformation())


General configuration for OpenCV 4.1.1 =====================================
  Version control:               4.1.1-2-gd5a58aa75

  Platform:
    Timestamp:                   2019-12-13T17:25:11Z
    Host:                        Linux 4.9.140-tegra aarch64
    CMake:                       3.10.2
    CMake generator:             Unix Makefiles
    CMake build tool:            /usr/bin/make
    Configuration:               Release

  CPU/HW features:
    Baseline:                    NEON FP16
      required:                  NEON
      disabled:                  VFPV3

  C/C++:
    Built as dynamic libs?:      YES
    C++ Compiler:                /usr/bin/c++  (ver 7.4.0)
    C++ flags (Release):         -fsigned-char -W -Wall -Werror=return-type -Werror=non-virtual-dtor -Werror=address -Werror=sequence-point -Wformat -Werror=format-security -Wmissing-declarations -Wundef -Winit-self -Wpointer-arith -Wshadow -Wsign-promo -Wuninitialized -Winit-self -Wsuggest-override -Wno-delete-non-vir